In [64]:
import pandas as pd  
import numpy as np  
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [69]:
picks = pd.read_parquet('../../Data/Historical_Picks_by_Week.parquet')
matchups = pd.read_parquet('../../Data/Historical_Player_Performance_Final.parquet')

In [70]:
matchups.head()

,season,week,owner,starter_fantasy_points,starter_fp_lag_1,starter_fp_lag_2,starter_fp_lag_3,starter_fp_lag_4,starter_fp_4wk_avg_lag1,starter_fp_8wk_avg_lag1,starter_fp_4wk_std_lag1,weekly_win_lag_1,weekly_loss_lag_1,weekly_tie_lag_1,weekly_win_lag_2,weekly_loss_lag_2,weekly_tie_lag_2,weekly_win_lag_3,weekly_loss_lag_3,weekly_tie_lag_3,weekly_win_lag_4,weekly_loss_lag_4,weekly_tie_lag_4,win_pct_4wk_lag1,cum_wins_lag_1,cum_losses_lag_1,win_pct_lag_1,cum_wins_lag_2,cum_losses_lag_2,win_pct_lag_2,cum_wins_lag_3,cum_losses_lag_3,win_pct_lag_3,cum_wins_lag_4,cum_losses_lag_4,win_pct_lag_4,win_pct_change_1wk,cum_wins_change_1wk,cum_points_lag_1,season_rank_lag_1,cum_points_lag_2,season_rank_lag_2,cum_points_lag_3,season_rank_lag_3,cum_points_lag_4,season_rank_lag_4,cum_points_4wk_avg_lag1
0,2023,1,Brigido,150.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023,2,Brigido,167.54,150.14,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,150.14,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023,3,Brigido,177.62,167.54,150.14,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,317.68,1.0,150.14,2.0,NaN,NaN,NaN,NaN,NaN
3,2023,4,Brigido,146.80,177.62,167.54,150.14,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,3.0,0.0,1.0,2.0,0.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,0.0,1.0,495.30,1.0,317.68,1.0,150.14,2.0,NaN,NaN,NaN
4,2023,5,Brigido,131.06,146.80,177.62,167.54,150.14,160.525,NaN,14.579359,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,0.0,1.0,3.0,0.0,1.0,2.0,0.0,1.0,1.0,0.0,1.0,-0.2,0.0,642.10,2.0,495.30,1.0,317.68,1.0,150.14,2.0,401.305


In [66]:
picks_full.shape

(510, 31)

In [67]:
matchups.shape

(510, 30)

In [51]:
train = picks_full.merge(matchups, 
                         how = 'left',
                         on = ['season', 'owner', 'week'])

In [60]:
train[(train['owner'] == 'Jose') & (train['season'] == 2023)]

,owner,season,week,2024_1,2024_2,2024_3,2024_4,2025_1,2025_2,2025_3,2025_4,2026_1,2026_2,2026_3,2026_4,2027_1,2027_2,2027_3,2027_4,2028_1,2028_2,2028_3,2028_4,early_round_picks,late_round_picks,near_term_picks,intermediate_term_picks,far_term_picks,total_future_picks,win_now_index,rebuild_index,starter_fantasy_points,starter_points_4gm_avg,best_expected_qb,best_expected_rb,best_expected_wr,best_expected_te,best_expected_flex,best_expected_sflex,best_expected_total_points,qb_count_10plus,rb_count_10plus,wr_count_12plus,te_count_8plus,rb_dropoff_2_to_3,wr_dropoff_3_to_4,flex_dropoff,sflex_qb_vs_nonqb_gap,lineup_efficiency_ratio
306,Jose,2023,1,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,5.0,5.0,4.0,14.0,0.117647,0.464286,125.14,125.140000,29.00,40.40,56.20,17.6,19.1,23.54,185.84,3,2,2,2,5.9,5.0,7.10,2.80,0.673375
307,Jose,2023,2,1.0,2.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,7.0,5.0,4.0,16.0,0.095238,0.406250,127.20,126.170000,26.34,23.70,60.86,11.6,26.0,25.20,173.70,2,4,5,2,0.9,5.1,1.50,4.38,0.732297
308,Jose,2023,3,1.0,2.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,7.0,5.0,4.0,16.0,0.095238,0.406250,179.44,143.926667,31.68,29.80,62.90,19.5,27.7,27.54,199.12,4,2,4,2,6.5,2.9,0.40,6.18,0.901165
309,Jose,2023,4,1.0,3.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,8.0,8.0,5.0,4.0,17.0,0.086957,0.382353,125.86,139.410000,19.64,16.60,72.70,18.2,28.2,15.22,170.56,2,0,4,2,2.2,1.8,0.30,-16.36,0.737922
310,Jose,2023,5,1.0,3.0,1.0,3.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,9.0,8.0,6.0,4.0,18.0,0.083333,0.388889,124.32,139.205000,23.24,20.80,53.18,29.2,36.8,19.72,182.94,2,1,3,2,4.6,2.8,1.10,-5.96,0.679567
311,Jose,2023,6,1.0,4.0,1.0,3.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,9.0,9.0,8.0,4.0,21.0,0.071429,0.380952,120.06,137.420000,19.34,14.90,55.40,3.1,14.6,18.32,125.66,2,0,3,0,1.4,5.1,6.10,-2.16,0.955435
312,Jose,2023,7,1.0,4.0,1.0,3.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,9.0,9.0,8.0,4.0,21.0,0.071429,0.380952,152.96,130.800000,41.86,30.50,42.70,15.5,22.5,19.44,172.50,3,1,2,2,2.4,0.0,3.50,20.96,0.886725
313,Jose,2023,8,1.0,4.0,1.0,3.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,9.0,9.0,8.0,4.0,21.0,0.071429,0.380952,132.26,132.400000,22.30,46.90,54.80,28.4,36.1,16.56,205.06,2,2,2,3,11.1,1.3,3.30,-6.10,0.644982
314,Jose,2023,9,1.0,5.0,1.0,3.0,2.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,10.0,10.0,9.0,4.0,23.0,0.064516,0.369565,168.24,143.380000,20.74,33.10,48.50,25.6,28.1,19.80,175.84,3,4,1,3,0.6,1.8,8.40,-8.86,0.956779
315,Jose,2023,10,1.0,5.0,1.0,3.0,2.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,10.0,10.0,9.0,4.0,23.0,0.064516,0.369565,136.36,147.455000,20.66,29.50,47.70,22.1,26.7,12.88,159.54,2,2,4,3,2.2,2.3,0.90,-1.44,0.854707


In [53]:
# Make a copy to avoid modifying the original
train_encoded = train.copy()

le = LabelEncoder()
train_encoded['owner'] = le.fit_transform(train_encoded['owner'])

In [54]:
train_encoded.head()

,owner,season,week,2024_1,2024_2,2024_3,2024_4,2025_1,2025_2,2025_3,2025_4,2026_1,2026_2,2026_3,2026_4,2027_1,2027_2,2027_3,2027_4,2028_1,2028_2,2028_3,2028_4,early_round_picks,late_round_picks,near_term_picks,intermediate_term_picks,far_term_picks,total_future_picks,win_now_index,rebuild_index,starter_fantasy_points,starter_points_4gm_avg,best_expected_qb,best_expected_rb,best_expected_wr,best_expected_te,best_expected_flex,best_expected_sflex,best_expected_total_points,qb_count_10plus,rb_count_10plus,wr_count_12plus,te_count_8plus,rb_dropoff_2_to_3,wr_dropoff_3_to_4,flex_dropoff,sflex_qb_vs_nonqb_gap,lineup_efficiency_ratio
0,0,2023,1,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,7.0,5.0,4.0,4.0,13.0,0.125000,0.461538,150.14,150.140,20.46,47.3,59.7,7.7,34.4,20.02,189.58,3,4,4,0,1.7,3.3,1.1,-3.94,0.791961
1,0,2023,2,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,6.0,4.0,4.0,14.0,0.111111,0.428571,167.54,158.840,36.56,53.5,56.9,14.6,34.3,27.12,222.98,3,4,3,3,7.7,3.0,0.7,8.66,0.751368
2,0,2023,3,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,4.0,4.0,16.0,0.090909,0.375000,177.62,165.100,31.68,27.9,65.4,23.4,33.3,12.20,193.88,2,4,5,3,1.4,1.3,4.3,3.78,0.916134
3,0,2023,4,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,4.0,4.0,16.0,0.090909,0.375000,146.80,160.525,28.94,30.1,69.7,15.0,23.9,13.56,181.20,2,3,3,2,2.1,4.9,9.9,2.34,0.810155
4,0,2023,5,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,4.0,4.0,16.0,0.090909,0.375000,131.06,155.755,23.86,20.2,54.0,27.7,15.5,5.80,147.06,1,1,1,1,0.1,1.4,13.9,-8.24,0.891201


In [55]:
X = train_encoded.drop(columns=['starter_fantasy_points', 'lineup_efficiency_ratio', 'owner'])
y = train_encoded['starter_fantasy_points']

In [56]:
train_encoded = train_encoded.sort_values(['season', 'week']).reset_index(drop=True)
X = X.loc[train_encoded.index]
y = y.loc[train_encoded.index]

In [57]:
tscv = TimeSeriesSplit(n_splits=3)  # you can adjust number of splits

for i, (train_idx, val_idx) in enumerate(tscv.split(X)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # ----------------------
    # 5. Train XGBoost
    # ----------------------
    model = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        #early_stopping_rounds=20,
        verbose=False
    )

    # ----------------------
    # 6. Evaluate
    # ----------------------
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print(f"Split {i+1} RMSE: {rmse:.2f}")

Split 1 RMSE: 18.84
Split 2 RMSE: 17.20
Split 3 RMSE: 19.38


In [30]:
last_season = 2025
latest_data = train_encoded[train_encoded['season'] == last_season].copy()


In [31]:
weeks = pd.DataFrame({"week": range(1, 18)})
owners = latest_data[['owner']].drop_duplicates()
grid_2026 = owners.merge(weeks, how='cross')
grid_2026['season'] = 2026

In [32]:
carry_columns = [
    '2024_1','2024_2','2024_3','2024_4',
    '2025_1','2025_2','2025_3','2025_4',
    '2026_1','2026_2','2026_3','2026_4',  # if these are future picks, keep projected values
    '2027_1','2027_2','2027_3','2027_4',
    '2028_1','2028_2','2028_3','2028_4',
    'early_round_picks','late_round_picks','near_term_picks',
    'intermediate_term_picks','far_term_picks','total_future_picks',
    'win_now_index','rebuild_index',
    'best_expected_qb','best_expected_rb','best_expected_wr','best_expected_te',
    'best_expected_flex','best_expected_sflex',
    'qb_count_10plus','rb_count_10plus','wr_count_12plus','te_count_8plus',
    'rb_dropoff_2_to_3','wr_dropoff_3_to_4','flex_dropoff','sflex_qb_vs_nonqb_gap'
]

In [33]:
X_2026 = grid_2026.merge(
    latest_data[['owner'] + carry_columns],
    on='owner',
    how='left'
)


In [34]:
X_2026.head()

,owner,week,season,2024_1,2024_2,2024_3,2024_4,2025_1,2025_2,2025_3,2025_4,2026_1,2026_2,2026_3,2026_4,2027_1,2027_2,2027_3,2027_4,2028_1,2028_2,2028_3,2028_4,early_round_picks,late_round_picks,near_term_picks,intermediate_term_picks,far_term_picks,total_future_picks,win_now_index,rebuild_index,best_expected_qb,best_expected_rb,best_expected_wr,best_expected_te,best_expected_flex,best_expected_sflex,qb_count_10plus,rb_count_10plus,wr_count_12plus,te_count_8plus,rb_dropoff_2_to_3,wr_dropoff_3_to_4,flex_dropoff,sflex_qb_vs_nonqb_gap
0,0,1,2026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,10.0,11.0,5.0,4.0,20.0,0.068966,0.325,28.58,26.3,65.2,18.4,20.3,14.20,2,3,3,2,0.4,7.0,5.2,0.48
1,0,1,2026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,10.0,11.0,5.0,4.0,20.0,0.068966,0.325,26.14,30.1,50.8,13.9,28.2,23.90,5,4,5,2,1.4,0.3,0.9,6.24
2,0,1,2026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,10.0,11.0,5.0,4.0,20.0,0.068966,0.325,19.72,26.7,34.3,11.5,17.6,14.04,4,2,2,2,2.6,0.1,2.5,4.92
3,0,1,2026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,10.0,11.0,5.0,4.0,20.0,0.068966,0.325,36.96,37.0,41.1,20.8,30.2,22.86,3,3,2,3,3.0,0.2,0.7,13.96
4,0,1,2026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,10.0,11.0,5.0,4.0,20.0,0.068966,0.325,36.76,36.0,55.4,21.7,37.7,36.28,3,2,2,5,6.6,0.0,0.4,5.46


In [35]:
leak_cols = ['starter_fantasy_points', 'lineup_efficiency_ratio', 'starter_points_4gm_avg', 'best_expected_total_points']
X_2026 = X_2026.drop(columns=[c for c in leak_cols if c in X_2026.columns])


In [36]:
le_owner = LabelEncoder()
le_owner.fit(train_encoded['owner'])
X_2026['owner'] = le_owner.transform(X_2026['owner'])

In [40]:
# 1. Drop non-feature columns that weren't in training, safely
X_2026_model = X_2026.drop(columns=['owner', 'season_2025', 'week_2025'], errors='ignore')

# 2. Make sure feature columns match training exactly
X_2026_model = X_2026_model[X.columns]  # reorder/select columns exactly as in X

# 3. Predict
y_2026_pred = model.predict(X_2026_model)

# 4. Add predictions back to X_2026
X_2026['predicted_fantasy_points'] = y_2026_pred

# 5. Optional: sort for readability
X_2026 = X_2026.sort_values(['owner', 'week']).reset_index(drop=True)

# 6. Display predictions
print(X_2026[['owner', 'week', 'predicted_fantasy_points']])


KeyError: "['starter_points_4gm_avg', 'best_expected_total_points'] not in index"